In [1]:
import os

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import transformers
from tensorflow import keras

# DistilBERT

In [2]:
%%time

from transformers import DistilBertTokenizer, TFDistilBertForQuestionAnswering

model_name = 'distilbert-base-cased-distilled-squad'

tokenizer = DistilBertTokenizer.from_pretrained(model_name)
distilbert = TFDistilBertForQuestionAnswering.from_pretrained(model_name)

CPU times: user 3.04 s, sys: 492 ms, total: 3.53 s
Wall time: 2.35 s


In [3]:
distilbert.summary()

Model: "tf_distil_bert_for_question_answering"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  65190912  
_________________________________________________________________
qa_outputs (Dense)           multiple                  1538      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 65,192,450
Trainable params: 65,192,450
Non-trainable params: 0
_________________________________________________________________


In [4]:
input_ids = tf.keras.layers.Input(shape=(tokenizer.max_len,),
                                  name='input_token',
                                  dtype='int32')

In [5]:
X = distilbert(input_ids)

In [6]:
model = tf.keras.Model(inputs=[input_ids], outputs=X)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_token (InputLayer)     [(None, 512)]             0         
_________________________________________________________________
tf_distil_bert_for_question_ ((None, 512), (None, 512) 65192450  
Total params: 65,192,450
Trainable params: 65,192,450
Non-trainable params: 0
_________________________________________________________________


In [8]:
question1 = ("What are they investigating?")
answer_text1 = ("However, antiviral drugs are not completely harmless, and the "
                "conditions under which individuals are willing to participate "
                "in a large-scale antiviral drug treatment program are as yet "
                "unknown. To this end we investigate the conditions under which "
                "a large-scale antiviral control program can prevent an "
                "epidimic, and we analyze at what point in an unfolding epidemic"
                " the risk of infection starts to outweigh the cost of "
                "antiviral treatment")

In [9]:
input_example = tf.constant(
    tokenizer.encode(question1, answer_text1, pad_to_max_length=True))[None, :]
outputs = model.predict(input_example)
start_scores, end_scores = outputs[:2]
answer_start = np.argmax(start_scores)
answer_end = np.argmax(end_scores)
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(
        input_example.numpy().tolist()[0][answer_start:answer_end]))

print(answer)

the conditions under which a large - scale antiviral control program can prevent an epidim


In [10]:
input_example.shape

TensorShape([1, 512])

In [11]:
model_path = 'models/distilbert/001'

In [12]:
%%time

model.save(model_path, save_format='tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/distilbert/001/assets
CPU times: user 16.8 s, sys: 392 ms, total: 17.2 s
Wall time: 17.3 s


In [13]:
model.inputs

[<tf.Tensor 'input_token:0' shape=(None, 512) dtype=int32>]

In [14]:
!saved_model_cli show --dir models/distilbert/001 --tag_set serve --signature_def serving_default

2020-05-10 20:07:37.093833: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-05-10 20:07:37.093907: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-05-10 20:07:37.093918: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
The given SavedModel SignatureDef contains the following input(s):
  inputs['input_token'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 512)
      name: serving_default_input_token:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['tf_distil_b

In [15]:
%%time

distilbert_model = tf.keras.models.load_model(
    os.path.join(os.path.abspath(''), model_path))

CPU times: user 7.69 s, sys: 429 ms, total: 8.12 s
Wall time: 8.03 s


In [16]:
question1 = ("What temperature kills HCoV-19?")
answer_text1 = ("It is stable in faeces at room temperature for at least "
                "1-2 days and can be stable in infected patients for up to "
                "4 days. Heat at 56 degrees Celsius kills the SARS coronavirus "
                "at around 10000 units per 15 minutes. Thus, temperature is an "
                "important factor in survival of COVID19 virus and this "
                "article focuses on understanding the relationship between "
                "temperature and COVID19 transmission from the data available "
                "between January-March 2020.")

input_example = tf.constant(
    tokenizer.encode(question1, answer_text1, pad_to_max_length=True))[None, :]
outputs = distilbert_model.predict(input_example)
start_scores, end_scores = outputs[:2]
answer_start = np.argmax(start_scores)
answer_end = np.argmax(end_scores)
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(
        input_example.numpy().tolist()[0][answer_start:answer_end]))

print(answer)

56 degrees Cels


### Model input definition from DistilBERT used by Tensorflow Serving

In [17]:
distilbert_model.inputs

[<tf.Tensor 'input_1:0' shape=(None, 512) dtype=int32>]

# Swivel CORD-19

In [18]:
swivel_path = 'models/swivel/001'

In [19]:
%%time

embed = hub.KerasLayer("https://tfhub.dev/tensorflow/cord-19/swivel-128d/3",
                       input_shape=[], 
                       dtype=tf.string)

INFO:absl:Using /tmp/tfhub_modules to cache modules.


CPU times: user 299 ms, sys: 200 ms, total: 499 ms
Wall time: 505 ms


In [20]:
model = tf.keras.Sequential()
model.add(embed)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               17301632  
Total params: 17,301,632
Trainable params: 0
Non-trainable params: 17,301,632
_________________________________________________________________


In [22]:
model.save(swivel_path)

INFO:tensorflow:Assets written to: models/swivel/001/assets


INFO:tensorflow:Assets written to: models/swivel/001/assets


In [23]:
model.inputs

[<tf.Tensor 'keras_layer_input:0' shape=(None,) dtype=string>]